In [ ]:
!pip install -U deep-translator
!pip install tiktoken openai langchain langchain-community langchain_openai faiss-cpu

In [ ]:
import os
import json
import tqdm
import pickle
import shutil
from pprint import pprint as pp
import re
from deep_translator import GoogleTranslator
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import openai
from google.colab import userdata
from google.colab import drive, files


transtator = GoogleTranslator(source='ru', target='en')
drive.mount('/content/drive')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Папка с рабочими файлами на ГуглДиск
path = r"/content/drive/MyDrive/data"
# Разархивирую ZIP файл с вакансиями
shutil.unpack_archive(os.path.join(path, 'vacancies_20240227.zip'), 'vacancies')
# Разархивирую ZIP файл с резюме
shutil.unpack_archive(os.path.join(path, 'resumes_20240227.zip'), 'resumes')

In [ ]:
#@title Вакансии
# Для удобства сохраняю все все вакансии в словарь, где ключи - это имена json файлов
dict_vacancies = {}
for filename in os.listdir('vacancies'):
    if filename.endswith('.json'):
        with open(os.path.join('vacancies', filename), 'r') as f:
            dict_vacancies[filename[:-5]] = json.load(f)

# Сохраню словарь на ГуглДиск
with open(os.path.join(path, 'vacancies_dict_base.pkl'), 'wb') as f:
    pickle.dump(dict_vacancies, f)

print('Количество вакансий:', len(dict_vacancies))

Количество вакансий: 100


In [ ]:
# Для первичной прикидки и поиска сходства вакансии
# с резюме, из вакансий беру только ключевые поля:
# 'position', 'mandatoryRequirements' и 'skills'.

# В вакансиях разные языки, но все Skills в резюме и вакансиях на английском изначально.
# Cобираю чанки по каждой вакансии с переводом на английский. Потом дешевле будет с ChatGPT.
# Если был английский - текст сохранится без изменений.

vacancies_chunks = []

for key, value in tqdm.tqdm(dict_vacancies.items()):
    # Только три ключевых поля 'position', 'skills', 'mandatoryRequirements'
    text_line = re.sub(r'\s+', ' ', f"""Position: {value['data']['position']}.
                       Skills: {(', ').join(value['skills'])}.
                       {(', ').join(value['data']['mandatoryRequirements'])}""")
    text_line = transtator.translate(text_line)
    vacancies_chunks.append(Document(page_content=text_line, metadata={'file': key}))

# Сохраню чанки по вакансиям на ГуглДиск ('position', 'skills', 'mandatoryRequirements')
with open(os.path.join(path, 'vacancies_chunks_base.pkl'), 'wb') as f:
    pickle.dump(vacancies_chunks, f)

100%|██████████| 100/100 [01:34<00:00,  1.06it/s]


In [ ]:
# Векторизирую и сохраняю базу
db_vacancies = FAISS.from_documents(vacancies_chunks, OpenAIEmbeddings())
db_vacancies.save_local(folder_path=path, index_name='db_vacancies_base')
len(db_vacancies.docstore._dict)

100

In [ ]:
#@title Резюме
# Любое резюме, например:
# resume = 'Будилов_Кирилл-ac4ecc8b-8da8-4441-b4de-a71c0805bdcd.json'
resume = 'f5-47ca-91b9-e24aa99197e8.json'

with open(os.path.join('resumes', resume), 'r') as f:
     resume = json.load(f)

In [ ]:
# Для первичного поиска сходства с вакансиями
# Из резюме поля только: 'title', 'skill_set', 'professional_roles ... name'
text_line = re.sub(r'\s+', ' ', f"""Position: {resume['title']}.
                   {resume['professional_roles'][0]['name']}.
                   Skills: {(', ').join(resume['skill_set'])}.""")
eng_line = transtator.translate(text_line)

eng_line # текст запроса на английском, по которому будем искать похожие вакансии

'Position: Java developer. Programmer, developer. Skills: Spring Framework, Java, Git, SQL, PostgreSQL, Hibernate ORM, Spring Boot, REST, OOP, Intellij IDEA, JUnit, Maven, Java Core, Docker, Kubernetes, Kafka, Spring, Hibernate, Swagger, GitHub, JDBC, JPA .'

In [ ]:
text_line # двуязычный вариант из резюме

'Position: Java-разработчик. Программист, разработчик. Skills: Spring Framework, Java, Git, SQL, PostgreSQL, Hibernate ORM, Spring Boot, REST, ООП, Intellij IDEA, JUnit, Maven, Java Core, Docker, Kubernetes, Kafka, Spring, Hibernate, Swagger, GitHub, JDBC, JPA.'

In [ ]:
# similarity_search_with_score
def files_and_scores(query, db):
    docs_and_scores = db.similarity_search_with_score(query, k=5)
    print('Scores:', [doc[1] for doc in docs_and_scores])
    print(', '.join([f"\nFile {i+1}: {doc[0].metadata['file']}.json"
                     for i, doc in enumerate(docs_and_scores)]))
    return [f"{doc[0].metadata['file']}.json" for doc in docs_and_scores]


db_vacancies = FAISS.load_local(folder_path=path,
                                embeddings=OpenAIEmbeddings(),
                                index_name='db_vacancies_base',
                                allow_dangerous_deserialization=True)

# Первая прикидка подходящих вакансий под резюме c выдачей файлов - подходящих вакансий

# База на английском. Запрос тоже на Aнглийском.
files = files_and_scores(eng_line, db_vacancies)

Scores: [0.12100253, 0.122104086, 0.13859051, 0.1414056, 0.1854817]

File 1: Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json, 
File 2: Java_разработчик-6e8ea5e3-47fb-4a15-84fa-6ab618dcbbe0.json, 
File 3: Java_разработчик-f6b48d5d-882e-4afd-8826-3d46b78342c6.json, 
File 4: Java_разработчик-654b79d7-6b07-4705-969d-41a9eef10503.json, 
File 5: Backend_разработчик-8a68226f-ea25-4b81-8eee-a0e8ea99fb2d.json


In [ ]:
# База на английском. Запрос с частями текста на Русском, как в резюме.
f2 = files_and_scores(text_line, db_vacancies)

Scores: [0.17278373, 0.1740806, 0.17588896, 0.17798966, 0.2235882]

File 1: Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json, 
File 2: Java_разработчик-f6b48d5d-882e-4afd-8826-3d46b78342c6.json, 
File 3: Java_разработчик-654b79d7-6b07-4705-969d-41a9eef10503.json, 
File 4: Java_разработчик-6e8ea5e3-47fb-4a15-84fa-6ab618dcbbe0.json, 
File 5: Kotlin_разработчик-6bcde28f-d5b1-4586-82f2-9731cd38c756.json


In [ ]:
print(f'Желаемая позиция и скилы соискателя (title, skill_set, professional_roles...name): \n{text_line}')
print('\nПодходящие вакансии (по полям: position, skills, mandatoryRequirements):')

# Словарь для полей вакансий, подходящиих под конкретное резюме
vacancies_similarity = {}

# Для примера, три вакансии из файлов по мин. Score
for filename in files[:3]:
    with open(os.path.join('vacancies', filename), 'r') as f:
            vacancy = json.load(f)
            # поля: 'position', 'mandatoryRequirements' и 'skills'
            out_line_base = re.sub(r'\s+', ' ', f"""Position: {vacancy['data']['position']}.
                       Skills: {(', ').join(vacancy['skills'])}.
                       {(', ').join(vacancy['data']['mandatoryRequirements'])}""")
            print(f'\nFile: {filename}')
            print(out_line_base)
            vacancies_similarity[filename[:-5]] = [out_line_base,]

### После выбора подходящих вакансий использую ChatGPT - как консультанта в помощь соискателю или HR по каждой вакансии. Предварительная поверхностная прикидка, т.к. думаю, что этот шаг должен быть в самом конце

In [ ]:
#@title Дополнительные поля из Резюме, которые буду использовать в запросе к ChatGPT
# Уже есть подготовленная строка eng_line (title, skill_set, professional_roles...name)
# Дополнительно из резюме возьму:
# skills, language, schedules, experience(position, description), area, relocation

add_resume_info = ''

if resume['skills']:
    add_resume_info += f"{resume['skills']}. "

if len(resume['language']):
    for lang in resume['language']:
        add_resume_info += f"{lang['name']} язык - уровень: {lang['level']['name']}. "

if resume['schedule']:
    add_resume_info += f"Желаемый график работы: {resume['schedule']['name']}. "

if resume['experience']:
    for n, place in enumerate(resume['experience']):
        add_resume_info += f"Опыт работы {n+1}: Позиция - {place['position']}. \
                             Описание работы {n+1}: {place['description']}. "

if resume['area']:
    add_resume_info += f"Локация - {resume['area']['name']}. "

if resume['relocation']:
    add_resume_info += f"Отношение к релокации - {resume['relocation']['type']['name']}."

add_resume_info = re.sub(r'\s+', ' ', add_resume_info)
# На английский
add_resume_info_en = GoogleTranslator(source='ru', target='en').translate(add_resume_info)


# Примечание, этот блок нужно еще подработать, так как в некоторых резюме
# разная структура полей иногда, например со словаря меняется на список словарей и т.д.

In [ ]:
#@title Дополнительные поля из выбранных Вакансий, которые буду использовать в запросе к ChatGPT
# В словаре vacancies_similarity уже есть: position, skills, mandatoryRequirements
# Дополнительно в словарь vacancies_similarity из выбранных вакансий запишу:
# experienceLevels, requiredLocation, projectTasks

# Для примера, три вакансии из файлов по мин. Score
for filename in files[:3]:
    with open(os.path.join('vacancies', filename), 'r') as f:
            vacancy = json.load(f)
            # поля: experienceLevels, requiredLocation, projectTasks
            out_line_2 = re.sub(r'\s+', ' ',
                         f"""Level: {(', ').join(vacancy['data']['experienceLevels'])}.
                         Location: {vacancy['data']['requiredLocation']}.
                         Tasks: {(', ').join(vacancy['data']['projectTasks'])}""")
            # print(f'\nFile: {filename}')
            # print(out_line_2)
            vacancies_similarity[filename[:-5]].append(out_line_2)

In [ ]:
# Имеем две части резюме
print('Резюме:')
print(f'Желаемая позиция и скилы:\n{text_line}') # title, skill_set, professional_roles...name
print(f'Опыт и др. информация от соискателя:\n{add_resume_info}') # skills, language, schedules, experience(position, description), area, relocation


vacancies_similarity_en = {}
# По выбранным трем вакансиям имеем:
print('\nВыбранные вакансии:')
for k, v in vacancies_similarity.items():
    print(f'\nFile: {k}.json')
    print(v[0]) # position, skills, mandatoryRequirements
    print(v[1]) # experienceLevels, requiredLocation, projectTasks

    # Выбранные вакансии на английский
    vacancies_similarity_en[k] = [transtator.translate(v[0]),
                                  transtator.translate(v[1])]

In [ ]:
#@title Подготовка запросов к ChatGPT


# функция добавления переходов на новую строку для удобства чтения
def format_newlines(text: str, max_len: int = 150) -> str:
    lines = text.splitlines()
    new_lines = []
    for line in lines:
        words = line.split()
        current_line = ""
        for word in words:
            if len(current_line + " " + word) > max_len:
                new_lines.append(current_line)
                current_line = ""
            current_line += f' {word}'
        new_lines.append(current_line)
    return "\n".join(new_lines)


# Стоимость запроса + ответ для "gpt-3.5-turbo-0125"
def print_tokens_count(completion):
    # global TOTAL_AMOUNT
    # "gpt-3.5-turbo-0125" - Input: $0.50 / 1M tokens - Output: $1.50 / 1M tokens - 07/03/2024 - https://openai.com/pricing
    price = 0.5 * completion.usage.prompt_tokens / 1e6 + 1.5 * completion.usage.completion_tokens / 1e6
    print(f'Использовано токенов: {completion.usage.prompt_tokens} + '
                                f'{completion.usage.completion_tokens} = '
                                f'{completion.usage.total_tokens}. '
                                f'Цена запроса + ответ: $ {price}\n')
    # TOTAL_AMOUNT += price


def answer_gpt(messages, temp=0.5):
    completion = openai.chat.completions.create(model="gpt-3.5-turbo-0125",
                                                messages=messages,
                                                temperature=temp)
    print_tokens_count(completion)
    return completion.choices[0].message.content

In [ ]:
# резюме у нас:
print(format_newlines(text_line))
print(format_newlines(add_resume_info))

 Position: Java-разработчик. Программист, разработчик. Skills: Spring Framework, Java, Git, SQL, PostgreSQL, Hibernate ORM, Spring Boot, REST, ООП,
 Intellij IDEA, JUnit, Maven, Java Core, Docker, Kubernetes, Kafka, Spring, Hibernate, Swagger, GitHub, JDBC, JPA.
 Моей целью в качестве разработчика на Java является использование своих технических навыков для поддержки ИТ-проектов и решения вопросов на благо
 компании и ее миссии. Имею исключительные обучающие способности с умением решать задачи. Являюсь командным игроком, способным к адаптации и открытым
 для изменений. Ищу отличную команду в крупной ИТ компании с интересными задачами по разработке, для развития своих компетенций в микросервисной
 архитектуре, базы данных, Spring Framework. Люблю спортивный бег, путешествия, художественную литературу, проводить время с семьей.. Русский язык -
 уровень: Родной. Желаемый график работы: Гибкий график. Опыт работы 1: Позиция - Java-разработчик. Описание работы 1: Разработал приложение серви

In [ ]:
#@title Какие то вопросы к ChatGPT, чтобы посмотреть как версия 3.5 вообще справляется )


def messages(vacancy):
    system = """You are a professional HR. You help candidates (applicants) to get a job for the chosen vacancy"""

    message_content = f"""Based on the candidate's resume, you have found a suitable vacancy.
    Candidate's desired position, candidate's skills, additional information about the candidate
    from the resume: {eng_line}. {add_resume_info_en}. Requirements for the candidate, according
    to the vacancy you have chosen: {vacancy}. Analyze the information from the candidate's
    resume and the requirements from the vacancy."""

    question = """Answer the questions:
    1. What should the candidate's attention be paid to when interviewing for a vacancy position?
    2. What are the strengths of the candidate for the chosen vacancy?
    3. What is the probability, as a percentage from 0 to 100, that a candidate will get a job for
    the chosen vacancy based on his skills and competencies?

    Answer all questions in Russian, with the exception of the words denoting the candidate's skills,
    if you write about them. So, answer in Russian, but if you use words denoting the candidate's skills,
    write skills in English."""

    return [{"role": "system", "content": system},
            {"role": "user", "content": f"{message_content}\n{question}"}]


# Запускаю вопрсы по каждой из трех выбранных вакансий
for k, v in vacancies_similarity_en.items():
    print('\n', '-'*50)
    print(f'Вакансия: {k}')
    response = answer_gpt(messages(vacancy = v[0] + v[1]), temp=0.3)
    print(format_newlines(response))


 --------------------------------------------------
Вакансия: Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c
Использовано токенов: 1036 + 552 = 1588. Цена запроса + ответ: $ 0.001346

 1. На собеседовании на вакансию кандидату стоит обратить внимание на следующие аспекты:
 - Опыт работы с MS SQL, PostgreSQL и Oracle Pl/SQL, так как это требуется по вакансии.
 - Навыки работы с CI/CD процессами на GitLab или аналогах.
 - Умение анализировать проблемы производительности и трассировать сложные ошибки.
 - Опыт работы в команде по методикам agile разработки (Scrum, Kanban и т. д.).
 - Знание Spring, Spring Boot, Spring Cloud, JPA и опыт работы с *nix командной строкой.
 - Навыки работы с Kubernetes, Docker и базами данных.

 2. Сильные стороны кандидата для выбранной вакансии:
 - Обширный опыт работы с Java, Spring Framework, Hibernate ORM и другими технологиями Java.
 - Успешное участие в разработке различных веб-приложений и сервисов.
 - Навыки разработки и оптимизации SQL запросо